In [3]:
import pandas as pd
df = pd.read_csv("../datasets_old/song_lyrics.csv")

In [ ]:
df[df["year"]]

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
1657,Genesis 1,misc,Holy Bible (KJV),2,82340,{},Genesis 1:1: In the beginning God created the ...,1636,en,en,en
46771,Matthew 27,misc,Holy Bible (KJV),90,2847,{},"Matthew 27:1: When the morning was come, all t...",49183,en,en,en
66885,Matthew 5,misc,Holy Bible (KJV),90,7902,{},"Matthew 5:1: And seeing the multitudes, he wen...",72235,en,en,en
79788,Matthew 14,misc,Holy Bible (KJV),90,1856,{},Matthew 14:1: At that time Herod the tetrarch ...,86455,en,en,en
82906,Vide,rap,Disiz,1,19094,{},"[Refrain x2]\nEt tu marches dans ta tête, ta t...",89980,fr,fr,fr
...,...,...,...,...,...,...,...,...,...,...,...
5129512,Snippet 02.04.2022,rap,4n Way,1,13,{},Я довёл её до слёз так было глубоко\nЯ мешаю г...,7872248,ru,ru,ru
5130545,Want Me,pop,SKM GEEK,1,4,"{""Eleanor Forte""}",[Verse 1]\nI wake up every day\nThinkin' 'bout...,7874199,en,en,en
5132125,FLEX,rap,UGO BO$$,1,3,"{""UGO BO\\$\\$"",""Yola Cray""}",UGO!\n\nNigga know I had to flex (Had to Flex)...,7877346,en,en,en
5133467,VIVIENNE LOCKETS,rap,Lil He77,2,16,{},[Chorus]\n(Yeah) tell me why you love me i've ...,7879900,en,en,en


In [1]:
import torch
print("Torch:", torch.__version__)
print("CUDA:", torch.version.cuda)

from torch.nn.attention import sdpa_kernel

print("Flash available:", sdpa_kernel.flash.is_available())
print("MemoryEff available:", sdpa_kernel.mem_efficient.is_available())

Torch: 2.3.0+cu121
CUDA: 12.1


AttributeError: 'function' object has no attribute 'flash'

In [5]:
!nvidia-smi

Sun Nov 16 08:10:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:81:00.0 Off |                  N/A |
| 32%   58C    P1            519W /  575W |   23411MiB /  32607MiB |     97%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
print("Flash SDP available:", torch.backends.cuda.is_flash_sdp_available())
print("MemEff SDP available:", torch.backends.cuda.is_mem_efficient_sdp_available())
print("Math SDP available:", torch.backends.cuda.is_math_sdp_available())

In [9]:
!uv pip install nltk
!uv pip install spacy
!python -m spacy download en_core_web_sm

Using Python 3.11.13 environment at: /workspace/.venv
Audited 1 package in 55ms
Using Python 3.11.13 environment at: /workspace/.venv
Resolved 34 packages in 350ms                                        
⠙ Preparing packages... (0/18)                                                  
⠙ Preparing packages... (0/18)-------------     0 B/21.82 KiB           
⠙ Preparing packages... (0/18)--------- 14.92 KiB/21.82 KiB         
⠙ Preparing packages... (0/18)--------- 14.92 KiB/21.82 KiB         
catalogue            ------------------------------     0 B/16.92 KiB
⠙ Preparing packages... (0/18)--------- 14.92 KiB/21.82 KiB         
catalogue            ------------------------------ 14.92 KiB/16.92 KiB
⠙ Preparing packages... (0/18)--------- 14.92 KiB/21.82 KiB         
catalogue            ------------------------------ 14.92 KiB/16.92 KiB
⠙ Preparing packages... (0/18)--------- 14.92 KiB/21.82 KiB         
catalogue            ------------------------------ 14.92 KiB/16.92 KiB
⠙ Preparing

In [6]:
import nltk
import os

# Force NLTK to download into /workspace/nltk_data
NLTK_DIR = "/workspace/nltk_data"
os.makedirs(NLTK_DIR, exist_ok=True)

nltk.data.path.append(NLTK_DIR)

nltk.download("punkt", download_dir=NLTK_DIR)
nltk.download("punkt_tab", download_dir=NLTK_DIR)
nltk.download("stopwords", download_dir=NLTK_DIR)

[nltk_data] Downloading package punkt to /workspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /workspace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /workspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# tfidf_train.py
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

TEXT_COL = "clean_lyrics"
ERA_COL = "song_era"

train_df = pd.read_csv("../datasets/train_split.csv")
val_df   = pd.read_csv("../datasets/val_split.csv")
test_df  = pd.read_csv("../datasets/test_split.csv")

2025-11-16 09:01:37.947290: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 09:01:38.019887: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 09:01:39.331947: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df["label_id"] = label_encoder.fit_transform(train_df[ERA_COL])
val_df["label_id"]   = label_encoder.transform(val_df[ERA_COL])
test_df["label_id"]  = label_encoder.transform(test_df[ERA_COL])

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=50000,
    ngram_range=(1,2),
    min_df=3
)

X_train = vectorizer.fit_transform(train_df[TEXT_COL])
X_val   = vectorizer.transform(val_df[TEXT_COL])
X_test  = vectorizer.transform(test_df[TEXT_COL])

y_train = train_df["label_id"].values
y_val   = val_df["label_id"].values
y_test  = test_df["label_id"].values


In [5]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 69393390 stored elements and shape (777865, 50000)>

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train_t = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_val_t   = torch.tensor(X_val.toarray(), dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val, dtype=torch.long)

train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128)

class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        return self.net(x)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MLP(X_train.shape[1], len(label_encoder.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

: 

In [6]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax'),
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

model.fit(
    X_train.toarray(), y_train,
    validation_data=(X_val.toarray(), y_val),
    epochs=10,
    batch_size=128,
    callbacks=[es]
)

model.save("models/tfidf_mlp.h5")
print("TF-IDF MLP model saved!")

/workspace/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1763284071.294998   37416 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
W0000 00:00:1763284071.328719   37416 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1763284071.515808   37416 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 29788 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5090, pci bus id:

InternalError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:Cast] name: 

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# -----------------------------------------
# CONFIG
# -----------------------------------------
CSV_PATH = "../datasets_old/song_lyrics_map_era.csv"
TEXT_COL = "clean_lyrics"       # ปรับตาม dataset ของคุณ
ERA_COL = "song_era"            # เช่น 1970s, 1980s...
TOP_K = 100                      # จำนวน top tokens ต่อ era
NGRAM_RANGE = (1, 2)            # 1-gram + 2-gram
MIN_DF = 5                      # กรอง token ที่โผล่น้อยเกินไป
OUTPUT_CSV = "era_top_tokens.csv"

# -----------------------------------------
# LOAD DATA
# -----------------------------------------
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=[TEXT_COL, ERA_COL])

eras = sorted(df[ERA_COL].unique())
print("Found eras:", eras)

# -----------------------------------------
# FIT TF-IDF ON ALL LYRICS
# -----------------------------------------
vectorizer = TfidfVectorizer(
        stop_words="english",
        max_features=80_000,
        ngram_range=(1, 2),
        min_df=3,
    lowercase=True,
)

print("Fitting TF-IDF on whole corpus...")
X = vectorizer.fit_transform(df[TEXT_COL])
vocab = vectorizer.get_feature_names_out()


Found eras: ['1970s', '1980s', '1990s', '2000s', '2010s', '2020s']
Fitting TF-IDF on whole corpus...


In [5]:

# -----------------------------------------
# EXTRACT TOP TOKENS FOR EACH ERA
# -----------------------------------------
rows = []

for era in eras:
    print(f"\nProcessing era: {era}")

    # เลือกเฉพาะเพลงใน era นี้
    mask = df[ERA_COL] == era
    X_era = X[mask.values]

    # คำนวณ mean TF-IDF ต่อ era
    tfidf_mean = X_era.mean(axis=0).A1

    # sort ค่า tfidf
    top_idx = tfidf_mean.argsort()[::-1][:TOP_K]

    for rank, idx in enumerate(top_idx, start=1):
        token = vocab[idx]
        score = tfidf_mean[idx]
        rows.append([era, rank, token, score])

# -----------------------------------------
# SAVE AS CSV
# -----------------------------------------
out_df = pd.DataFrame(rows, columns=["era", "rank", "token", "tfidf_score"])
out_df.to_csv(OUTPUT_CSV, index=False)

print("\nSaved:", OUTPUT_CSV)
print(out_df.head(20))


Processing era: 1970s

Processing era: 1980s

Processing era: 1990s



Processing era: 2000s

Processing era: 2010s

Processing era: 2020s

Saved: era_top_tokens.csv
      era  rank  token  tfidf_score
0   1970s     1   love     0.033963
1   1970s     2     oh     0.023225
2   1970s     3     ll     0.018694
3   1970s     4    don     0.018387
4   1970s     5   baby     0.018288
5   1970s     6   know     0.018110
6   1970s     7     ve     0.017015
7   1970s     8   just     0.016926
8   1970s     9   time     0.014809
9   1970s    10    got     0.014571
10  1970s    11   come     0.013604
11  1970s    12    way     0.012663
12  1970s    13   like     0.012525
13  1970s    14   yeah     0.012464
14  1970s    15    man     0.012262
15  1970s    16  gonna     0.011946
16  1970s    17    let     0.011760
17  1970s    18   away     0.011368
18  1970s    19    day     0.011296
19  1970s    20  night     0.010747
